In [1]:
# default_exp utils.visualization

In [54]:
# export

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.cm as cmx
import matplotlib.patches as patches

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap, LogNorm

from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture

from datetime import datetime
from pathlib import Path
from typing import Optional, Dict, Tuple
from collections.abc import Iterable

# Configs

import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

## visualization

> Methods for visualization purposes, specially for clustering-related results.

## Utils

In [49]:
# export

# Util. method for saving pyplot plots and validate output directory

def _save_plot(out_path: str,  file_name: str, file_format: Optional[str]='png'):
    """
    Store pyplot figures veryfing containing directories
    """
    path = Path(out_path)
    
    if not path.exists():
        path.mkdir(exist_ok=True)
        
    now = datetime.now()
    ts = datetime.timestamp(now)
    
    plt.savefig(f"{str(path/file_name)}-{ts}.{file_format}", dpi=100, format=file_format)

In [22]:
# export

# Generate Mixed Gaussian Model with k components
def generate_distributions(feature_vectors, k):
    """
    Generate Mixed Gaussian mixture model with k components
    :param feature_vectors: Data
    :param k: Number of components to get GMM
    :return: sklearn GMM
    """
    gmm = GaussianMixture(k, covariance_type="diag")
    gmm.fit(feature_vectors)
    
    return gmm

In [ ]:
# export

def setup_data(cluster):
    """
    Get decision boundaries by means of np.meshgrid
    
    :return: Tuple (vectors, centroids, X component of mesghgrid, Y component of meshgrid, )
    """
    feature_vectors, _, centroids, _, kmeans = cluster
    # Step size of the mesh. Decrease to increase the quality of the VQ.
    h = .2     # point in the mesh [x_min, x_max]x[y_min, y_max].

    # Plot the decision boundary. For that, we will assign a color to each
    x_min, x_max = feature_vectors[:, 0].min() - 1, feature_vectors[:, 0].max() + 1
    y_min, y_max = feature_vectors[:, 1].min() - 1, feature_vectors[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    return feature_vectors, centroids, xx, yy, Z

In [39]:
# export

def setup_meshgrid(cluster, lim):
    samples, _, medoid_ids, kmedoids = cluster
    medoids = np.array([samples[i] for i in medoid_ids])
    h = .5     # point in the mesh [x_min, x_max]x[y_min, y_max].

    # Plot the decision boundary. For that, we will assign a color to each
    #x_min, x_max = samples[:, 0].min() - 1, samples[:, 0].max() + 1
    # y_min, y_max = samples[:, 1].min() - 1, samples[:, 1].max() + 1
    
    xx, yy = np.meshgrid(np.arange(-lim, lim, h), np.arange(-lim, lim, h)) # np.meshgrid(np.arange(-10, 10, h), np.arange(-10, 10, h))
    
    Z = kmedoids.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    
    return samples, medoids, xx, yy, Z

In [37]:
# export

# Change to providing it the actual samples from the distribution instead of sampling from it

def gmm_kl(gmm_p, gmm_q, n_samples=10**5):
    """
    Compute the KL-divergence value for 2 given
    Gaussian Mixture models
    
    :param gmm_p: 1st GMM model
    :param gmm_q: 2nd GMM model
    
    :return: Float indicating KL-divergence value
    """
    # From https://stackoverflow.com/a/26079963/5768407
    X = gmm_p.sample(n_samples)[0]
    log_p_X = gmm_p.score_samples(X)
    log_q_X = gmm_q.score_samples(X)
    return log_p_X.mean() - log_q_X.mean()

## Configuration

In [19]:
params = {
    "result_plots_path": "/tf/main/dvc-ds4se/results/images"
}

## Plotting methods

Plotting data distribution

In [40]:
# export

def plot_gmm_2d(clusters: Dict, model: str, out_path: str, export = False):
    plt.figure(figsize=(12, 8))
    # Create 2x2 sub plots
    gs = gridspec.GridSpec(1, 2)
    plt.clf()
    for i, model in enumerate(clusters):
        samples, _, medoid_ids, _ = clusters[model]
        gmm = generate_distributions(samples, len(medoid_ids))

        # Plot Data
        plt.subplot(gs[0, i])
        visualize_2D_gmm(samples, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, i)
        
    if export:
        _save_plot(out_path, f"2D_GMM_{model}")
    plt.show()

In [38]:
# export

def plot_overlapping_features(clusters: Dict, 
                              out_path: str, colmap: Optional[Dict]={0: 'c.', 1: 'm.', 2: 'y.'},
                              lim: Optional[int]=10, export: Optional[bool]=False):
    """
    Plot the distribution for 2 sets of samples
    
    :param clusters: Dict containing cluster info. for 2 distributions
    :param out_path: Str indicating the location for storing generated figures
    :param colmap: Dict specifying colors for plots
    :param lim: int
    :param export: Bool indicating whether or not to store generated figure
    """
    plt.figure(figsize=(8, 8))
    plt.title(f'Overlapping features for {" and ".join(clusters.keys())} samples. \n')
    for i, model in enumerate(clusters):
        samples, _, _, _ = clusters[model]
        plt.plot(samples[:, 0], samples[:, 1], colmap[i], markersize = 4)
        
        plt.xlabel('T-SNE Dim 1')
        plt.ylabel('T-SNE Dim 2')
        
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    plt.legend(clusters.keys())
    plt.grid(True)

    if export:
        _save_plot(out_path, f'overlapping_features_{"_".join(clusters.keys())}')

In [12]:
# export

# From http://www.itzikbs.com/gaussian-mixture-model-gmm-3d-point-cloud-classification-primer
def visualize_2D_gmm(points, w, mu, stdev, id, c, export=False):
    '''
    plots points and their corresponding gmm model in 2D
    Input: 
        points: N X 2, sampled points
        w: n_gaussians, gmm weights
        mu: 2 X n_gaussians, gmm means
        stdev: 2 X n_gaussians, gmm standard deviation (assuming diagonal covariance matrix)
    Output:
        None
    '''
    n_gaussians = mu.shape[1]
    N = int(np.round(points.shape[0] / n_gaussians))
    # Visualize data
    axes = plt.gca()
    colors = cmx.Accent(np.linspace(0, 1, n_gaussians)) if id == 'human' else cmx.Paired(np.linspace(0, 1, n_gaussians))
    for i in range(n_gaussians):
        idx = range(i * N, (i + 1) * N)
        for j in range(4):
            axes.add_patch(
                patches.Ellipse(mu[:, i], width=(j+1) * stdev[0, i], height=(j+1) *  stdev[1, i], alpha = 0.4, fill=True, color=colors[i]))
    plt.xlabel('T-SNE Dim 1')
    plt.ylabel('T-SNE Dim 2')

In [34]:
# export

def plot_gmm(cluster: Dict, model: str, c, out_path: str, export: Optional[bool]=True):
    """
    Plot Gaussian Mixture models for an individual set of samples
    
    :param cluster:
    :param model:
    :param c:
    :param out_path:
    :param export: Bool indicating whether or not store the generated figure
    """
    plt.figure(figsize = (8, 8))
    plt.title(f'Gaussian Mixture Models for {model} samples.\n')
    
    samples, _, medoid_ids, _ = cluster
    plt.plot(samples[:, 0], samples[:, 1], c, markersize = 4, zorder = -1)
    gmm = generate_distributions(samples, len(medoid_ids))

    # Plot Data
    visualize_2D_gmm(samples, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, c)
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    plt.legend([model])
    plt.grid(True)
    
    if export:
        _save_plot(out_path, f"gmm_{model}")

In [10]:
# export

def plot_kmedoids(cluster, model,
                  cmap = ListedColormap(["whitesmoke", "lightgrey", 'darkgrey', 'dimgrey', 'black']),
                  cm = 'c.', lim = 10, export = True):
    plt.figure(figsize=(8, 8))
    samples, centroids, xx, yy, Z = setup_data(cluster, lim)
    _, crit_ids = gen_criticisms(samples, centroids)
    plt.imshow(
        Z, interpolation='nearest',
        extent = (xx.min(), xx.max(), yy.min(), yy.max()),
        cmap = cmap, aspect = 'auto', origin = 'lower'
    )
    plt.plot(samples[:, 0], samples[:, 1], cm, markersize = 4)
    plt.scatter(
        centroids[:, 0], centroids[:, 1],
        marker = 'o', s = 64, linewidths = 3,
        color = 'b', zorder = 10
    )
    for crit_id in crit_ids:
        plt.scatter(
            samples[crit_id, 0], samples[crit_id, 1],
            marker = 'x', s = 64, linewidths = 3, color = 'r',
            zorder = 10
        )
    plt.title(
        f'K-Medoids clustering via\n'
        f'(PCA & T-SNE - reduced data) for {model} testbed\n'
    )
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    plt.xlabel('T-SNE Dim 1')
    plt.ylabel('T-SNE Dim 2')
    plt.legend([model, 'prototypes', 'criticisms'])
    plt.grid(True)
    
    if export:
        plt.savefig(f'/tf/data/graphics/k-medoids_{model}.png', dpi=100, format='png')

In [42]:
# export

def plot_overlapping_gmm(clusters: Dict, out_path: str,
                         colmap: Optional = {0: 'c.', 1: 'm.', 2: 'y.'},
                         lim: Optional[int]=10, export: Optional[bool]=True):
    """
    Plot 2 distributions
    
    :param clusters: Dict containing clusters data for 2 distributions
    :param colmap: Dict indicating colors for datapoints
    :param lim:
    :param export: Bool indicating whether or not save the generated figure
    
    """
    plt.figure(figsize = (8, 8))
    plt.title(f'Gaussian Mixture Models for {" and ".join(clusters.keys())} samples.\n')

    gmms = {}
    cs = ['c', 'g', 'm', 'y']
    sets = []
    for i, model in enumerate(clusters):
        samples, _, medoid_ids, _ = clusters[model]
        plt.plot(samples[:, 0], samples[:, 1], colmap[i], markersize = 4, zorder = -1)
        gmm = generate_distributions(samples, len(medoid_ids))
        gmms[model] = gmm
        
        # Plot Data
        visualize_2D_gmm(samples, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, cs[i])
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    plt.legend(clusters.keys())
    plt.grid(True)
    
    if export:
        _save_plot(out_path, f"overlapping_gmm_{'_'.join(clusters.keys())}")

In [15]:
# export

def plot_features(clusters, export = True):
    plt.figure(figsize=(12, 8))
    # Create 2x2 sub plots
    gs = gridspec.GridSpec(2, 2)
    plt.clf()
    
    plt.subplot(gs[1, 0])
    colmap = {0: 'co', 1: 'mo'}
    plt.title('Overlapping Models\nCyan denotes Human Methods and\nMagenta denotes Generator Samples')
    for i, model in enumerate(clusters):
        samples, _, _, _ = clusters[model]
        plt.plot(samples[:, 0], samples[:, 1], colmap[i], markersize=2)
        plt.xlim(-10, 10)
        plt.ylim(-10, 10)
        plt.xlabel('T-SNE Dim 1')
        plt.ylabel('T-SNE Dim 2')
    
    plt.subplot(gs[1, 1])
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    gmms = {}
    cs = ['c', 'm']
    sets = []
    for i, model in enumerate(clusters):
        samples, _, medoid_ids, _ = clusters[model]
        gmm = generate_distributions(samples, len(medoid_ids))
        gmms[model] = gmm
        
        # Plot Data
        visualize_2D_gmm(samples, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, cs[i])
    human_to_model_kl = gmm_kl(gmms['human'], gmms['model'])
    model_to_human_kl = gmm_kl(gmms['model'], gmms['human'])
    plt.title(f'Human to Model KL Divergence: {human_to_model_kl}\nModel to Human KL Divergence: {model_to_human_kl}')
    
    plt.show()
    
    if export:
        if not os.path.exists('images/'): os.mkdir('images/')
        plt.savefig('images/1D_GMM_demonstration.png', dpi=100, format='png')

In [ ]:
# export


def plot_gmm_1d(clusters, export = False):
    plt.figure(figsize=(12, 8))
    # Create 2x2 sub plots
    gs = gridspec.GridSpec(1, 2)
    plt.clf()
    for i, model in enumerate(clusters):
        samples, _, medoid_ids, _ = clusters[model]
        gmm = generate_distributions(samples, len(medoid_ids))
        x_min, x_max = samples[:, 0].min(), samples[:, 0].max()

        # Plot Data
        plt.subplot(gs[0, i])

        delta = 10
        x = np.linspace(x_min - delta, x_max + delta, 1000).reshape(500,2)
        logprob = gmm.score_samples(x)
        pdf = np.exp(logprob)
        plt.plot(x, pdf, '-k')
        
    if export:
        if not os.path.exists('images/'): os.mkdir('images/')
        plt.savefig('images/1D_GMM_demonstration.png', dpi=100, format='png')
        
    plt.show()

In [ ]:
# export

def plot_overlapping_and_gmms(clusters, out_path: str, export: Optional[bool]=True):
    """
    Plot the overlapping distribution of 2 sample sets
    
    :param clusters: Dict. containing clusters info.
    :param export: Bool indicating whether or not store the generated figure
    
    """
    plt.figure(figsize=(12, 8))
    # Create 2x2 sub plots
    gs = gridspec.GridSpec(2, 2)
    plt.clf()
        
    plt.subplot(gs[1, 0])
    colmap = {0: 'co', 1: 'mo'}
    plt.title('Overlapping Models\nCyan denotes Human Methods and\nMagenta denotes Generator Samples')
    for i, model in enumerate(clusters):
        samples, _, _, _ = clusters[model]
        plt.plot(samples[:, 0], samples[:, 1], colmap[i], markersize=2)
        plt.xlim(-10, 10)
        plt.ylim(-10, 10)
        plt.xlabel('T-SNE Dim 1')
        plt.ylabel('T-SNE Dim 2')
    
    plt.subplot(gs[1, 1])
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    gmms = {}
    cs = ['c', 'm']
    sets = []
    for i, model in enumerate(clusters):
        samples, _, medoid_ids, _ = clusters[model]
        gmm = generate_distributions(samples, len(medoid_ids))
        gmms[model] = gmm
        
        # Plot Data
        visualize_2D_gmm(samples, gmm.weights_, gmm.means_.T, np.sqrt(gmm.covariances_).T, model, cs[i])
        
    # Compute Kl divergence
    human_to_model_kl = gmm_kl(gmms['human'], gmms['model'])
    model_to_human_kl = gmm_kl(gmms['model'], gmms['human'])
    plt.title(f'Human to Model KL Divergence: {human_to_model_kl}\nModel to Human KL Divergence: {model_to_human_kl}')
    
    plt.show()
    
    if export:
        _save_plot(out_path, f"")
        plt.savefig('images/1D_GMM_demonstration.png', dpi=100, format='png')

In [51]:
# export

def plot_clusters(clusters: Iterable, out_path: str, export: Optional[bool]=True,
                  clustering_type: Optional[str]='K-medoids'):
    plt.figure(figsize=(12, 8))
    # Create 2x2 sub plots
    gs = gridspec.GridSpec(2, 2)
    plt.clf()
    for i, cluster in enumerate(clusters):
        # Setup data to be plotted
        feature_vectors, centroids, xx, yy, Z = setup_data(cluster)
        
        logger.info('Generated setup data (decision boundaries).')
        
        # Plot data
        plt.subplot(gs[0, i])
        plt.imshow(Z, interpolation='nearest',
               extent=(xx.min(), xx.max(), yy.min(), yy.max()),
               cmap=plt.cm.Paired,
               aspect='auto', origin='lower')

        plt.plot(feature_vectors[:, 0], feature_vectors[:, 1], 'k.', markersize=2)
        # Plot the centroids as a white X
        plt.scatter(centroids[:, 0], centroids[:, 1],
                    marker='x', s=169, linewidths=3,
                    color='w', zorder=10)
        plt.title(f'{clustering_type} Clustering\n'
                  '(PCA & T-SNE - reduced data)\n'
                  'Centroids are marked with white cross')
        plt.xlim(xx.min(), xx.max())
        plt.ylim(yy.min(), yy.max())
        
    plt.subplot(gs[1, :])
    colmap = {0: 'b.', 1: 'r.'}
    plt.title('Blue denotes Human Methods and Red denotes Generator Samples')
    for i, cluster in enumerate(clusters):
        feature_vectors, _, _, _, _ = cluster
        plt.plot(feature_vectors[:, 0], feature_vectors[:, 1], colmap[i], markersize=10)
        
    if export:
        _save_plot(out_path, f"clustered_datapoints_")
        plt.savefig('images/feature_vectors_scatter_plot.png', dpi=100, format='png')
    plt.show()

In [55]:
from nbdev.export import notebook2script
notebook2script()

Converted 0.0_mgmnt.prep.i.ipynb.
Converted 0.1_mgmnt.prep.conv.ipynb.
Converted 0.3_mgmnt.prep.bpe.ipynb.
Converted 0.6_mgmnt.prep.nltk.ipynb.
Converted 0.7_mgmnt.prep.files_mgmnt.ipynb.
Converted 0.8_mgmnt.prep.bpe_tokenization.ipynb.
Converted 1.0_exp.i.ipynb.
Converted 1.1_exp.info-[inspect].ipynb.
Converted 1.1_exp.info.ipynb.
Converted 1.2_exp.csnc.ipynb.
Converted 1.2_exp.gen.code.ipynb.
Converted 1.3_exp.csnc_python.ipynb.
Converted 10.0_utils.clusterization.ipynb.
Converted 10.1_utils.visualization.ipynb.
Converted 2.0_repr.codebert.ipynb.
Converted 2.0_repr.i.ipynb.
Converted 2.1_repr.codeberta.ipynb.
Converted 2.1_repr.roberta.train.ipynb.
Converted 2.2_repr.roberta.eval.ipynb.
Converted 2.3_repr.word2vec.train.ipynb.
Converted 2.6_repr.word2vec.eval.ipynb.
Converted 2.7_repr.distmetrics.ipynb.
Converted 2.8_repr.sentence_transformers.ipynb.
Converted 3.1_mining.unsupervised.traceability.eda.ipynb.
Converted 3.2_mining.unsupervised.eda.traceability.d2v.ipynb.
This cell doesn